In [1]:
from main import *
from visualization import *

import bct
import hdbscan
from sklearn.manifold import TSNE

%matplotlib inline
plt.rcParams['font.size'] = 6

# Setting paths

In [2]:
top_directory = '/media/anleg84/Backup Plus/CERVO/Data/Networks2024/'
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'

In [3]:
atlas = MapzebrainAtlas(atlas_path)
centroids = np.concatenate([atlas.regionCentroids['left'], atlas.regionCentroids['right']], axis=0)
centroids[:, 1] = 974 - centroids[:, 1]

In [4]:
datasets = get_datasets(top_directory, keywords=['920nm'], exclude=[])
print(len(datasets))

22


# Delta F / F stack

This notebook goes through every raw data folder and generates a 'detrended' `.tif` file for each calcium imaging plane. These detrended videos are used in the `Figure4` notebook to transform calcium imaging frames into the brain atlas to visualize coactivation event cluster centroids.

In [6]:
import numpy as np
import multiprocessing as mp
from scipy.signal import detrend

def process_pixel_series(pixel_series):
    bp = np.linspace(0, len(pixel_series), 3, endpoint=True).astype('int')
    processed_series = detrend(gaussian_filter1d(pixel_series, 2), bp=bp)
    return processed_series

def worker(pixel_series_chunk):
    return [process_pixel_series(series) for series in pixel_series_chunk]

def parallel_detrend_stack(stack_data, N_processes=None):
    T, X, Y = stack_data.shape
    reshaped_stack = stack_data.reshape(T, X*Y)
    if N_processes is None:
        N_processes = mp.cpu_count()
    pixel_chunks = np.array_split(reshaped_stack.T, N_processes)
    pool = mp.Pool(processes=N_processes)
    processed_chunks = pool.map(worker, pixel_chunks)
    pool.close()
    pool.join()
    processed_stack = np.concatenate(processed_chunks).T.reshape(T, X, Y)
    return processed_stack

In [ ]:
for folder in datasets:
    if not any(identify_files(folder, ['dff_corrected_', '.tif'])):
        files = identify_files(folder, ['corrected', '.tif'])
        for file in tqdm(files, file=sys.stdout):
            stack = load_stack(folder + file).astype('float32')
            stack = parallel_dff_stack(stack, N_processes=30)
            stack[stack < 0] = 0
            stack -= np.min(stack)
            stack /= np.max(stack)
            stack *= 65535
            save_stack(folder + 'dff_' + file, stack)

 95%|████████████████████████████████████████▉  | 20/21 [13:22<00:40, 40.63s/it]